# 8 - Hyperparameter & Model Selection

Consider the objective function of Linear SVM:

\begin{equation}
J(\bar\theta) = \frac{1}{2}\|\bar\theta\|_2^2 + C\sum_{i=1}^{N} \max\{0, 1 - y^{(i)}\bar{\theta}\cdot \bar{x}^{(i)}\}
\end{equation}

Given training data, there are two sets of unknowns:
- Model parameter(s): the coefficients $\bar\theta$, which are found by fitting the model, 
- Hyperparameter(s): the regularization strength (or its inverse, $C$), which has to be specified prior to the fitting the model.

**Note:** Model parameters can be determined through a learning algorithm (e.g., SGD, or calling `fit(X, y)`. But how do we select the best hyperparameters?

In [ ]:
#@title Run this cell to download preprocessed data (features + labels). { display-mode: "form" }
!pip install -U wget
!rm -rf preprocessed
!mkdir preprocessed

import wget
wget.download('https://github.com/shengpu1126/BDSI2019-ML/raw/master/preprocessed/data.npz', 'preprocessed/data.npz')

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import metrics, exceptions

In [ ]:
with np.load('preprocessed/data.npz') as f:
    X = f['X']
    y = f['y']
    feature_names = f['feature_names']

In [ ]:
#@title Run this cell to define the preprocessing functions. { display-mode: "form" }
#@markdown - `impute_missing_values(X)`
#@markdown - `normalize_feature_matrix(X)`

def impute_missing_values(X):
    """
    For each feature column, impute missing values  (np.nan) with the 
    population mean for that feature.
    
    Args:
        X: np.array, shape (N, d). X could contain missing values
    Returns:
        X: np.array, shape (N, d). X does not contain any missing values
    """
    from sklearn.impute import SimpleImputer
    return SimpleImputer().fit_transform(X)

def normalize_feature_matrix(X):
    """
    For each feature column, normalize all values to range [0, 1].

    Args:
        X: np.array, shape (N, d).
    Returns:
        X: np.array, shape (N, d). Values are normalized per column.
    """
    from sklearn.preprocessing import MinMaxScaler
    return MinMaxScaler().fit_transform(X)

In [ ]:
X = impute_missing_values(X)
X = normalize_feature_matrix(X)

In [ ]:
print('First 10 labels:', y[:10])
print('First 2 feature vectors:\n', X[:2])

## Pipeline v0

In [ ]:
clf = SVC(kernel='linear') # use default arguments here
clf.fit(X, y)
y_pred = clf.predict(X)
accuracy = metrics.accuracy_score(y, y_pred)
print('Accuracy:', accuracy)

In [ ]:
#@title What's wrong with Pipeline v0? How do we fix it? { display-mode: "form" }
answer =  #@param {type:"raw"}

## Pipeline v1

In [ ]:
# Split data into train (80%) and test (20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=3)

In [ ]:
clf = SVC(kernel='linear')
clf.fit(...)
y_pred = clf.predict(...)
accuracy = metrics.accuracy_score(...)
print('test accuracy:', accuracy)

In [ ]:
#@title What's wrong with Pipeline v1? How do we fix it? { display-mode: "form" }
answer =  #@param {type:"raw"}

## Pipeline v2

In [ ]:
for C in [1e-1, 1, 1e1]:
    clf = SVC(kernel='linear', C=C)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print('C={}'.format(C), '\t', 'test accuracy:', accuracy)
# in the end, pick the C with best test accuracy

In [ ]:
#@title What's wrong with Pipeline v2? How do we fix it? { display-mode: "form" }
answer =  #@param {type:"raw"}

---
## Pipeline v3: Hyperparameter Selection via Cross validation

We will select hyperparameters using 5-fold cross-validation (CV) on the training data. Specifically, we will select the hyperparameters that lead to the best average validation performance across all five folds. The result of hyperparameter selection often depends on the choice of performance measure. 

### (a)
First, implement the function `cv_performance(clf, X, y, metric='accuracy', k=5)` to calculate cross-validated performance given a classifier and (training) data `X, y`.

When dividing the data into folds for CV, you should try to keep the class proportions (ratio of positive to negative labels) roughly the same across folds. You may employ the following class for splitting the data: `sklearn.model_selection.StratifiedKFold()`. For consistency of results, do not shuffle points when using this function (i.e., do not set `shuffle=True`).

In [ ]:
#@title Why might it be beneficial to maintain class proportions across folds? { display-mode: "form" }
answer = ' ' #@param {type:"raw"}

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

def cv_performance(clf, X, y, metric='accuracy', k=5):
    """
    Splits the training data X and the labels y into k-folds and runs k-fold
    cross-validation: for each fold i in 1...k, trains a classifier on
    all the data except the ith fold, and tests on the ith fold.
    
    Calculates the k-fold cross-validation performance metric for classifier
    clf by averaging the performance across folds.
    
    Input:
        clf: a classifier
        X: (n,d) array of feature vectors, where n is the number of examples
           and d is the number of features
        y: (n,) array of binary labels
        k: an int specifying the number of folds (default=5)
        metric: string specifying the performance metric
    
    Returns:
        average validation performance across the k folds
    """
    # TODO: Implement this function
    
    ## HINT: You may find the StratifiedKFold from sklearn.model_selection
    ## to be useful

    # Put the performance of the model on each fold in the scores array
    scores = []

    # And return the average performance across all fold splits.
    return np.array(scores).mean()

### (b)
Now implement the `select_C_linear_SVM(X, y, metric='accuracy', k=5, C_range=[])` function to choose a value of C for a linear SVM, using 5-fold cross validation on the training data with specified metric. This function should call the `cv_performance` function that you implemented above, passing in instances of `SVC(kernel='linear', C=C)` with a range of values for C chosen in powers of 10 (between $10^{−3}$ and $10^3$).

In [ ]:
def select_C_linear_SVM(X, y, metric='accuracy', k=5, C_range=[]):
    """
    Sweeps different settings for the hyperparameter of a linear SVM, 
    calculating the k-fold CV performance for each setting of C on 
    training data X, y.
    
    Input:
        X: (n,d) array of feature vectors, where n is the number of examples
        and d is the number of features
        y: (n,) array of binary labels
        k: int specifying the number of folds (default=5)
        metric: string specifying the performance metric
        C_range: an array with C values to be searched over
    
    Returns:
        The value of C parameter for a linear SVM that maximizes the
        average 5-fold CV performance.
    """
    # TODO: Implement this function
    #HINT: You should be using your cv_performance function here
    #to evaluate the performance of each SVM
    return 0.0

### (c)
If you need to train a final model, which performance measure would you optimize for when choosing $C$? Explain your choice.

Using the training data and functions implemented here, find the best setting for $C$ for your chosen performance measure. Using the best $C$ value you found, train a SVM on the entire training set `X_train, y_train`. Report the performance of this SVM on the test data `X_test, y_test` for each metric below.

In [ ]:
## TODO
metric_list = ["accuracy", "f1_score", "auroc", "precision", "sensitivity", "specificity"]


In [ ]:
## TODO
# Select the best C based on your chosen metric

# Train a model using best C on entire training set

# Evaluate classifier performance on test set


| Metric |  Best C | Test Performance |
|------|------|------|
| accuracy | ? | ? |
| F1-score | ? | ? |
| AUROC    | ? | ? |
| Precision | ? | ? |
| Sensitivity | ? | ? |
| Specificity | ? | ? |

### (d)
The L0-norm of $\bar{\theta} \in \mathbb{R}^d$ is defined as:
\begin{align*}
    \|\bar{\theta}\|_0 = \sum_{j=1}^{d} [[\theta_j \neq 0]]
\end{align*}
where $[[a \neq 0]]$ is 0 if $a$ is 0 and 1 otherwise.

Plot the L0-norm of $\bar{\theta}$, the parameter vector learned by the SVM, for each value of $C$. Use the complete training data `X_train, y_train`, i.e, don't use cross-validation for this part. 

In [ ]:
from sklearn.svm import SVC, LinearSVC
def get_classifier(kernel='linear', penalty='l2', C=1.0, gamma=0.0, class_weight=None):
    """
    Return a linear/rbf kernel SVM classifier based on the given
    penalty function and regularization parameter c.
    """
    if penalty == 'l2':
        if kernel == 'linear':
            return SVC(kernel='linear', C=C, class_weight=class_weight)
        elif kernel == 'rbf':
            return SVC(kernel='rbf', C=C, gamma=gamma, class_weight=class_weight)
    elif penalty == 'l1':
        return LinearSVC(penalty='l1', C=C, dual=False, max_iter=20000, class_weight=class_weight)
    
    raise ValueError('Error: unsupported configuration')

def plot_weight(X, y, penalty, C_range):
    """
    Takes as input the training data X and labels y and plots the L0-norm
    (number of nonzero elements) of the coefficients learned by a classifier
    as a function of the C-values of the classifier.
    """
    print("Plotting the number of nonzero entries of the parameter vector as a function of C")
    norm0 = []
    
    ### Solution
    for C in C_range:
        clf = get_classifier(kernel='linear', C=C, penalty=penalty)
        clf.fit(X, y)
        w = clf.coef_
        w = np.squeeze(np.asarray(w))
        norm0.append(np.linalg.norm((w),ord=0))
    ### Solution
    
    # This code will plot your L0-norm as a function of C
    fig, ax = plt.subplots()
    plt.plot(C_range, norm0)
    plt.xscale('log')
    plt.legend(['L0-norm'])
    plt.xlabel("Value of C")
    plt.ylabel("Norm of theta")
    plt.title('Norm-'+penalty+'_penalty.png')
    return fig


In [ ]:
# L2 regularization
fig = plot_weight(X_train, y_train, 'l2', np.logspace(-5, 2, 8))
plt.ylim(0-1, 40+1)
plt.grid()
plt.show()

In [ ]:
# L1 regularization
fig = plot_weight(X_train, y_train, 'l1', np.logspace(-5, 2, 8))
plt.ylim(0-1, 40+1)
plt.grid()
plt.show()

In [ ]:
#@title Describe any interesting trends you observe. { display-mode: "form" }
answer =  #@param {type:"raw"}